# Preprocess Required for Generating Train Data: 

In [14]:
import urllib.request
import time
import sys
import getopt
import pandas as pd
import numpy as np
import pickle


In [15]:
%run ../utils.ipynb

In [16]:
embSize = 200

ftrain='../data/EUADR_target_disease.csv'
# Replace with path of word embdding file   
#wefile = "/mnt/admin/GDA_backup/Dataset/embeddings/PubMed-and-PMC-w2v.bin"
wefile = "/mnt/admin/GDA_backup/Dataset/embeddings/wi PubMed-and-PMC-w2v.bin"
random_seed=1331

# Read Data

In [18]:
Tr_sent_contents, Tr_entity1_list, Tr_entity2_list, Tr_sent_lables = dataRead_befree_EUADR(ftrain)

Tr_word_list, Tr_d1_list, Tr_d2_list  = get_wordList_and_distances_befree(Tr_sent_contents)

print ("train_size", len(Tr_word_list))


Input File Reading
train_size 355


## Prepare Lable Matrix

In [19]:
# Y : is positive association 
# N: is negative association
label_dict = {'FA':0, 'NA':1,'PA':1,'SA':1}

Y_t = mapLabelToId_befree_EUADR(Tr_sent_lables, label_dict)
Y_train = np.zeros((len(Y_t), 2))
for i in range(len(Y_t)):
    Y_train[i][Y_t[i]] = 1.0

# Generate Word and Position Embedding Vectors

### Word Embedding

In [20]:
word_dict, word_to_id, id_to_word = word_mapping(Tr_word_list)

print( "word dictonary length", len(word_dict))

# Word Embedding
word_vectors = readWordEmb(word_dict,id_to_word,word_to_id, wefile, embSize,limit=2000000)
W_train =   mapWordToId(Tr_word_list, word_to_id)


Found 1355 unique words (10062 in total)
word dictonary length 1355
Reading word vectors
Loaded 2000000 pretrained embeddings.
number of unknown word in word embedding 129


### Position Embedding

In [21]:
d1_dict = makeDistanceList([Tr_d1_list])
d2_dict = makeDistanceList([Tr_d2_list])
d1_train = mapWordToId_list(Tr_d1_list, d1_dict)
d2_train = mapWordToId_list(Tr_d2_list, d2_dict)


### Pad Embdding Vectors

In [22]:
train_sent_lengths= findSentLengths([Tr_word_list])

sentMax =max(max(train_sent_lengths[:]))

W_train, d1_train, d2_train = paddData([W_train, d1_train, d2_train ], sentMax,padd_num=100000) 


# Save Prepared Data as Pickle File

In [24]:
with open('../data/pickles/befree_EUADR_2class_PubMed-and-PMC-w2v.pickle', 'wb') as handle:
    pickle.dump(W_train, handle)
    pickle.dump(d1_train, handle)
    pickle.dump(d2_train, handle)    
    pickle.dump(Y_train, handle)
    pickle.dump(Tr_word_list, handle)
    pickle.dump(word_vectors, handle)
    pickle.dump(word_dict, handle)
    pickle.dump(d1_dict, handle)
    pickle.dump(d2_dict, handle)
    pickle.dump(label_dict, handle) 
    pickle.dump(sentMax, handle)
    
    